# OSPO Project Health Data

# Tableau Functions

**These functions will contain the code to generate data for Tableau by calling the common functions that I use to gather the data and interpret it**

In [3]:
# Sustains and Keeps Up with Contributions Function

def sustain_prs_by_repo_tableau(repo_id, repo_name, org_name, start_date, end_date, engine):

    import pandas as pd
    from common_functions import sustain_prs_by_repo_data
    # Delete import below after testing
    from common_functions import sustain_prs_by_repo_graph

    error_num, error_text, pr_sustainDF, title, title_color, interpretation, risk, risk_num = sustain_prs_by_repo_data(repo_id, repo_name, org_name, start_date, end_date, engine)

    # error_num == 0 - ok to generate graph. If error_num == -1, there aren't enough PRs to generate the graph.
    # pr_sustainDF: is the dataframe containing the data you need to graph
    # title: this is the title for the top of the chart
    # title_color: The title text should be displayed in this color
    # interpretation: This text should appear at the bottom of the graph
    # risk and risk num: You can ignore these variables

    if error_num == 0:
        # Add code here to generate what you need for Tableau. 

        # for testing and validation below. You can delete the lines below when you start generating yours graphs
        # this creates my .png chart, and you might want to look at it to see how I've accessed the dataframe. 
        sustain_risk_num, sustain_risk = sustain_prs_by_repo_graph(repo_id, repo_name, org_name, start_date, end_date, engine)
        # print the dataframe so that you know what you have
        print('Sustain Dataframe\n', pr_sustainDF, '\n\n')

In [4]:
# Contributor Risk Function

def contributor_risk_tableau(repo_id, repo_name, org_name, start_date, end_date, engine):

    print('contributor_risk_tableau not implemented yet')



In [5]:
# Timely Responses Function

def response_time_tableau(repo_id, repo_name, org_name, start_date, end_date, engine):

    print('response_time_tableau not implemented yet')


In [6]:
# Actively Maintained - Regular Releases Function

def activity_release_tableau(repo_name, org_name, start_date, end_date, repo_api):

    print('activity_release_tableau not implemented yet')


# Main Script

**The script below will generate all of the data using the functions above in addition to some common functions**

In [7]:
from common_functions import augur_db_connect, get_dates, get_commits_by_repo
from common_functions import repo_api_call, fork_archive

six_months = 180  # Default to one year of data
year = 365   # Default to one year of data

engine = augur_db_connect()

start_date, end_date = get_dates(year)
six_start_date, six_end_date = get_dates(six_months)

commit_threshold = 60 # 90 but use 1500 for testing

repo_list_commits = get_commits_by_repo(six_start_date, six_end_date, engine)

top = repo_list_commits.loc[repo_list_commits['count'] > commit_threshold]

# Testing - Delete this line later
i = 0

for index, repo in top.iterrows():

    repo_id = repo['repo_id']
    repo_name = repo['repo_name']
    repo_path = repo['repo_path']
    org_name = repo_path[11:(len(repo_path)-1)]

    print('Processing:', org_name, repo_name, repo_path, repo_id, repo['count'])

    try:
        repo_api = repo_api_call(repo_name, org_name)
    except:
        print('Cannot process API calls for:', org_name, repo_name, repo_path, repo_id)

    is_fork, is_archived = fork_archive(repo_name, org_name, engine)

    # Only gather data from repos that aren't forks or archived
    if is_fork == False and is_archived == False:
        sustain_prs_by_repo_tableau(repo_id, repo_name, org_name, start_date, end_date, engine)
        contributor_risk_tableau(repo_id, repo_name, org_name, start_date, end_date, engine)
        response_time_tableau(repo_id, repo_name, org_name, start_date, end_date, engine)
        activity_release_tableau(repo_name, org_name, start_date, end_date, repo_api)

    # Testing - Delete these lines later
    if i > 2:
        break
    else:
       i+=1

Processing: vmware captive-web-view github.com/vmware/ 28028 61
contributor_risk_tableau not implemented yet
response_time_tableau not implemented yet
activity_release_tableau not implemented yet
Processing: vmware-tanzu velero-plugin-for-vsphere github.com/vmware-tanzu/ 28046 62

Sustaining and keeping up with contributions for velero-plugin-for-vsphere 
from '2020-02-29' to '2021-02-28' 
saved as /Users/dafoster/gitrepos/augur-pivotal/code/health_scripts/./output/2021-02/vmware-tanzu/velero-plugin-for-vsphere/velero-plugin-for-vsphere_sustains_pr.png
AT RISK - Number of months in the past 6 months with > 10% of PRs not closed 2 

Sustain Dataframe
    yearmonth                  repo_name  repo_id  closed_total  all_total  \
0    2020-02  velero-plugin-for-vsphere    28046             8          8   
1    2020-03  velero-plugin-for-vsphere    28046            36         36   
2    2020-04  velero-plugin-for-vsphere    28046            16         16   
3    2020-05  velero-plugin-for-v